<hr/>

# Introduction to Data Science
**Tamás Budavári** - budavari@jhu.edu <br/>

- Classification exercises
- Cross-validation

<hr/>

<h1><font color="darkblue">Classification</font></h1>

- Based on a **training set** of labeled points, assign class labels to unknown vectors in the **query set**.  

> **Training set**
><br>
><br>
>$\qquad \displaystyle T = \big\{ (x_i, C_i) \big\}$ 
><br>
><br>
> where $x_i\in \mathbb{R}^d$ are feature sets and $C_i$ are the known class memberships

<nbsp/>

> **Query set**
><br>
><br>
>$\qquad \displaystyle Q = \big\{ x_i \big\}$ 
><br>
><br>
> where $x_i\in \mathbb{R}^d$ consist of the kind of features in $T$

- And again, $x_i$ are not real vectors but **feature sets** of a bunch of scalars in general

### Bayes with Covariance Matrix

- Estimate the full covariance matrix for the classes

>$\displaystyle {\cal{}L}_{\!\boldsymbol{x}}(C_k) =  G(\boldsymbol{x};\mu_k, \Sigma_k)$
><br>
> Handles correlated features well

- Consider binary problem with 2 classes - using Bayes' rule

>$ \displaystyle \frac{P(C_1|x)}{P(C_2|x)} = \frac{\pi_1}{\pi_2}\cdot \frac{{\cal{}L}_{\!\boldsymbol{x}}(C_1)}{{\cal{}L}_{\!\boldsymbol{x}}(C_2)} $

> Taking the negative logarithm, we compare
><br><br>
>$\displaystyle (x\!-\!\mu_1)^T\,\Sigma_1^{-1}(x\!-\!\mu_1) + \ln\,\lvert\Sigma_1\lvert $ 
> <br> vs.
><br>
>$\displaystyle (x\!-\!\mu_2)^T\,\Sigma_2^{-1}(x\!-\!\mu_2) + \ln\,\lvert\Sigma_2\lvert $
><br>
><br>
> If the difference is higher/lower than a threshold (based on the priors), we classify $x$ accordingly

- This is called **Quadratic Discriminant Analysis**

### Same Covariance Matrix

- When $\Sigma_1=\Sigma_2=\Sigma$, the quadratic terms cancel from the difference
 
>$\displaystyle (x\!-\!\mu_1)^T\,\Sigma^{-1}(x\!-\!\mu_1) $ 
>$\displaystyle -\ (x\!-\!\mu_2)^T\,\Sigma^{-1}(x\!-\!\mu_2) $

- Hence this is called **Linear Discriminant Analysis**

> Fewer parameters to estimate during the learning process
> <br>
> Good, if we don't have enough data, for example...
> <br>
> Think linear vs quadratic fitting and how you decide between those

### Exercise: QDA 

- Use the provided [training](Class-Train.csv) and [query](Class-Query.csv) sets to perform classification

> **Training** set consists of 3 columns of ($x_i$, $y_i$, $C_i$)
> <br>
> **Query** set only has 2 columns of ($x_i$, $y_i$)



> #### Best class?
>$\displaystyle \max_k \big[\ P(C_k|x)\ \big]$
>
>$\displaystyle \max_k \big[\ \pi_k {\cal{}L}_x(C_k)\ \big]$
>
>$\displaystyle \min_k \big[ -\ln\pi_k - \ln{\cal{}L}_x(C_k)\ \big]$

> #### Multivariate normal
>$\displaystyle {\cal{}L}_x(C_k) = \frac{1}{\sqrt{\lvert2\pi\Sigma_k\rvert}} \exp\left(-\frac{1}{2} (x\!-\!\mu_k)^T \Sigma_k^{-1} (x\!-\!\mu_k)\right)$
>
> Hence,
>
>$\displaystyle \min_k \Big[ \frac{1}{2} (x\!-\!\mu_k)^T \Sigma_k^{-1} (x\!-\!\mu_k) + \frac{1}{2}\ln\lvert\Sigma_k\rvert -\ln\pi_k \ \Big]$

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
class MyQDA(object):
    """ Template for a classifier
    """
    def fit(self,X,C):
        self.param = dict()
        # your code here
        klist = np.unique(C)
        for k in klist:
            members = C==k
            prior = members.sum() / C.size  # X.shape[0]
            S = X[members, :]
            mu = S.mean(axis=0)
            Z = (S-mu).T  # centered column vectors
            cov = Z @ Z.T / (Z.shape[1] - 1)
            self.param[k] = (mu, cov, prior)
        return self

    def predict(self,Y):
        Cpred = -1 * np.ones(Y.shape[0])
        # use linalg.det(matrix)
        # and linalg.inv(matrix)
        for i in range(Cpred.size):
            d2min, kbest = 1e99, None
            for k in self.param:
                mu, cov, prior = self.param[k]
                diff = (Y[i, :] - mu).T  # 2D column vector
                d2 = diff.T @ np.linalg.inv(cov) @ diff / 2
                d2 += np.log(np.linalg.det(cov)) / 2 - np.log(prior)
                if d2 < d2min:
                    kbest = k
                    d2min = d2
            Cpred[i] = kbest
        return Cpred
    
cls = MyQDA()
cls.fit(X,C)
Cpred = cls.predict(Q)
np.unique(Cpred)

[0. 1.]


array([0., 1.])

In [23]:
# reference implementation
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

D = np.loadtxt('files/Class-Train.csv', delimiter=',')
Q = np.loadtxt('files/Class-Query.csv', delimiter=',')
X, C = D[:,0:2], D[:,2]

Cpred = MyQDA().fit(X,C).predict(Q)
Cskit =   QDA().fit(X,C).predict(Q)

print ('Number of different estimates:', (Cpred!=Cskit).sum())

[0. 1.]
Number of different estimates: 0


<h1><font color="darkblue">Cross-Validation</font></h1>

- How to evaluate the quality of estimator?

> $k$-NN method's parameter affects the results

- We saw on the IRIS data that 1-NN was overfitting

> We discussed excluding the point itself

### Partitions of the Training set

- Random complementary subsets 

> Train on a larger subset, test on a small
> <br>
> Multiple rounds to decrease variance

### Leave-One-Out

- For each point, we train on the others and test

> Testing on $n$ points requires $n$ trainings

- Expensive!

### A Relaxed Variant

- $k$-fold cross-validation 

> 1. Create $k$ partitions of equal sizes, e.g., $k=2$ yields two subsets
> 2. Pick a single partition and train on the other $(k\!-\!1)$ 
> 3. Repeat for all $k$ partitions - requires $k$ trainings

- Leave-One-Out is a special case with $k=n$


### Exercise: Cross-Validation

- Evaluate QDA on the [training](files/Class-Train.csv) set using 2-fold cross-validation

> 1. What is the fraction of correct estimates? 
> 2. What is the uncertainty of that fraction?
 
> The **training** set consists of 3 columns of ($x_i$, $y_i$, $C_i$)


In [29]:
Dc = D.copy()
# randomize and split to D1 + D2
np.random.seed(seed=42)
np.random.shuffle(Dc)
split = Dc.shape[0] // 2  # or int(Dc[:,0].size/2)
D1, D2 = Dc[:split,:], Dc[split:,:]

# train on one, estimate on the other
for i, (T, Q) in enumerate([(D1, D2), (D2, D1)]):
    print(i, T.shape, Q.shape)
    X, C = T[:, :2], T[:, 2]
    CPred = MyQDA().fit(X,C).predict(Q[:, :2])
    Ctrue = Q[:, 2]

0 (156, 3) (157, 3)
[0. 1.]
1 (157, 3) (156, 3)
[0. 1.]


### Done already?

- Visualize the results in the 2D features space
- Make these simple codes run faster 


### Unhomework

- Implement LDA and compare to sklearn

>1. Write code without using sklearn 
>2. Apply to [training](Class-Train.csv) and [query](Class-Query.csv) sets 
>3. Compare your results to sklearn's 

- Perform 10-fold cross-validation of *MyQDA* on [this](Class-Train.csv) file

>1. Write code without using `sklearn` 
>2. Calculate average number of good classifications 
>3. Compare to sklearn 

In [ ]:
from sklearn.model_selection import cross_val_score
clf = QDA()
cross_val_score(clf, X,C, cv=10)

### What does this mean?

In [ ]:
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=10, shuffle=False) 

for k, (train, test) in enumerate(k_fold.split(X)):
    clf.fit(X[train],C[train])
    Cpred = clf.predict(X[test])
    print (k, ':\t', (C[test]==Cpred).sum() / float(test.size),
        '  =  ', clf.score(X[test],C[test]) )